In [ ]:
import numpy as np
import pandas as pd
import random
import re
import sklearn
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.translate.meteor_score import meteor_score, single_meteor_score

import nltk
from nltk.translate import meteor
from nltk import word_tokenize
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

To perform the translation experiment, the following analysis was perform to generate the METEOR scores for Gemini's translations.

# Part I: Romanian

In [ ]:
# Load data and handle empty rows
df_romanian = pd.read_csv("METEOR_Romanian.csv")
df_romanian = df_romanian.dropna()

# Preview data
df_romanian.head()

,Prompt,Google Translate - Romanian,Eng - Romanian (Gemini)
0,Suggest the ingredients needed for Roasted Swe...,Legume:\n\n2-3 cartofi dulci medii (in functie...,Legume:\n\n2-3 cartofi dulci medii (în funcție...
1,"Create an ingredient list in English, Chinese,...",Dovlecei de vară și fritjere ricotta cu sos de...,Friggerele de dovlecei cu ricotta și sos pican...
2,I want to go to the farmers market for a two c...,Supă de legume de primăvară\n\n2 linguri ulei ...,Ciorbă de legume de primăvară\n\nIngrediente:\...
3,Tomorrow is Christmas Eve. What ingredients sh...,Prăjirea festivă de curcan cu sos de afine\n\n...,Pui curcan festiv la tiga cu sos de merisoare ...
4,What ingredients do I need for a dish typical ...,fel principal:\n\nTurcia: Aceasta este vedeta ...,Meniul principal:\n\nCurcan: Vedeta mesei! Ale...


In [ ]:
# Select the columns containing the references and hypotheses (text to evaluate)
df_romanian['Google-Romanian'] = df_romanian['Google Translate - Romanian'] # Have the Google Translate translations as ground truth
df_romanian['Gemini-Romanian'] = df_romanian['Eng - Romanian (Gemini)'] # Have the Gemini translations as the ones being tested

# Preview first few rows
df_romanian[:5]

,Prompt,Google Translate - Romanian,Eng - Romanian (Gemini),Google-Romanian,Gemini-Romanian
0,Suggest the ingredients needed for Roasted Swe...,Legume:\n\n2-3 cartofi dulci medii (in functie...,Legume:\n\n2-3 cartofi dulci medii (în funcție...,Legume:\n\n2-3 cartofi dulci medii (in functie...,Legume:\n\n2-3 cartofi dulci medii (în funcție...
1,"Create an ingredient list in English, Chinese,...",Dovlecei de vară și fritjere ricotta cu sos de...,Friggerele de dovlecei cu ricotta și sos pican...,Dovlecei de vară și fritjere ricotta cu sos de...,Friggerele de dovlecei cu ricotta și sos pican...
2,I want to go to the farmers market for a two c...,Supă de legume de primăvară\n\n2 linguri ulei ...,Ciorbă de legume de primăvară\n\nIngrediente:\...,Supă de legume de primăvară\n\n2 linguri ulei ...,Ciorbă de legume de primăvară\n\nIngrediente:\...
3,Tomorrow is Christmas Eve. What ingredients sh...,Prăjirea festivă de curcan cu sos de afine\n\n...,Pui curcan festiv la tiga cu sos de merisoare ...,Prăjirea festivă de curcan cu sos de afine\n\n...,Pui curcan festiv la tiga cu sos de merisoare ...
4,What ingredients do I need for a dish typical ...,fel principal:\n\nTurcia: Aceasta este vedeta ...,Meniul principal:\n\nCurcan: Vedeta mesei! Ale...,fel principal:\n\nTurcia: Aceasta este vedeta ...,Meniul principal:\n\nCurcan: Vedeta mesei! Ale...


In [ ]:
# Define a function to clean the rows to perform NLP
def clean_text(text):
  # Remove newlines
  text_without_newline = re.sub('\n', ' ', text)

  # Remove all punctuation, including parentheses, colons, and others
  text_without_brackets = re.sub(r'[^\w\s]', '', text_without_newline)

  # Replace multiple consecutive spaces with a single space
  text_cleaned = re.sub(r'\s{2,}', ' ', text_without_brackets)

  return text_cleaned.lower()

# Clean the Romanian translations
df_romanian['Google-Romanian'] = df_romanian['Google-Romanian'].apply(lambda x: clean_text(x))
df_romanian['Gemini-Romanian'] = df_romanian['Gemini-Romanian'].apply(lambda x: clean_text(x))

# Preview first few rows
df_romanian[:5]

,Prompt,Google Translate - Romanian,Eng - Romanian (Gemini),Google-Romanian,Gemini-Romanian
0,Suggest the ingredients needed for Roasted Swe...,Legume:\n\n2-3 cartofi dulci medii (in functie...,Legume:\n\n2-3 cartofi dulci medii (în funcție...,legume 23 cartofi dulci medii in functie de ma...,legume 23 cartofi dulci medii în funcție de mă...
1,"Create an ingredient list in English, Chinese,...",Dovlecei de vară și fritjere ricotta cu sos de...,Friggerele de dovlecei cu ricotta și sos pican...,dovlecei de vară și fritjere ricotta cu sos de...,friggerele de dovlecei cu ricotta și sos pican...
2,I want to go to the farmers market for a two c...,Supă de legume de primăvară\n\n2 linguri ulei ...,Ciorbă de legume de primăvară\n\nIngrediente:\...,supă de legume de primăvară 2 linguri ulei de ...,ciorbă de legume de primăvară ingrediente 2 li...
3,Tomorrow is Christmas Eve. What ingredients sh...,Prăjirea festivă de curcan cu sos de afine\n\n...,Pui curcan festiv la tiga cu sos de merisoare ...,prăjirea festivă de curcan cu sos de afine ing...,pui curcan festiv la tiga cu sos de merisoare ...
4,What ingredients do I need for a dish typical ...,fel principal:\n\nTurcia: Aceasta este vedeta ...,Meniul principal:\n\nCurcan: Vedeta mesei! Ale...,fel principal turcia aceasta este vedeta emisi...,meniul principal curcan vedeta mesei alegeți u...


In [ ]:
# Define a function that tokenizes and calculates METEOR scores for each row
def calculate_meteor_R(row):
    # Tokenize the hypothesis text
    hypothesis_tokens = word_tokenize(row['Gemini-Romanian'])
    # Tokenize the reference text
    reference_tokens = [word_tokenize(row['Google-Romanian'])]
    # Calculate and return METEOR score
    return meteor_score(reference_tokens, hypothesis_tokens)

# Apply the function to each row of the dataframe
df_romanian['METEOR_score'] = df_romanian.apply(calculate_meteor_R, axis=1)

# Display the dataframe with the new METEOR scores
df_romanian

,Prompt,Google Translate - Romanian,Eng - Romanian (Gemini),Google-Romanian,Gemini-Romanian,METEOR_score
0,Suggest the ingredients needed for Roasted Swe...,Legume:\n\n2-3 cartofi dulci medii (in functie...,Legume:\n\n2-3 cartofi dulci medii (în funcție...,legume 23 cartofi dulci medii in functie de ma...,legume 23 cartofi dulci medii în funcție de mă...,0.571311
1,"Create an ingredient list in English, Chinese,...",Dovlecei de vară și fritjere ricotta cu sos de...,Friggerele de dovlecei cu ricotta și sos pican...,dovlecei de vară și fritjere ricotta cu sos de...,friggerele de dovlecei cu ricotta și sos pican...,0.686471
2,I want to go to the farmers market for a two c...,Supă de legume de primăvară\n\n2 linguri ulei ...,Ciorbă de legume de primăvară\n\nIngrediente:\...,supă de legume de primăvară 2 linguri ulei de ...,ciorbă de legume de primăvară ingrediente 2 li...,0.715339
3,Tomorrow is Christmas Eve. What ingredients sh...,Prăjirea festivă de curcan cu sos de afine\n\n...,Pui curcan festiv la tiga cu sos de merisoare ...,prăjirea festivă de curcan cu sos de afine ing...,pui curcan festiv la tiga cu sos de merisoare ...,0.591768
4,What ingredients do I need for a dish typical ...,fel principal:\n\nTurcia: Aceasta este vedeta ...,Meniul principal:\n\nCurcan: Vedeta mesei! Ale...,fel principal turcia aceasta este vedeta emisi...,meniul principal curcan vedeta mesei alegeți u...,0.691787
5,Design an ingredient list for a hearty Irish s...,Tocană irlandeză consistentă\n\nIngrediente:\n...,Tochitura irlandeză consistentă\n\nIngrediente...,tocană irlandeză consistentă ingrediente 1 lin...,tochitura irlandeză consistentă ingrediente 1 ...,0.585060
6,Create a recipe for a Pad Thai using only ingr...,Easy Pad Thai\n\nIngrediente:\n1 lingura ulei ...,Pad Thai ușor\n\nIngrediente:\n1 lingură de ul...,easy pad thai ingrediente 1 lingura ulei veget...,pad thai ușor ingrediente 1 lingură de ulei ve...,0.656568
7,"Recently, I just finished strolling through th...",Tofu imputit:\nUlei vegetal pentru prajit\nAmi...,Tofu Puțuros:\n\nUlei vegetal pentru prăjit\nA...,tofu imputit ulei vegetal pentru prajit amidon...,tofu puțuros ulei vegetal pentru prăjit amidon...,0.634325
8,My Singaporean friend is coming to visit me at...,Orez cu pui din Haina\n\nPentru puiul Hainanes...,Pui Hainanez cu Orez\n\nPentru puiul Hainanez:...,orez cu pui din haina pentru puiul hainanese 1...,pui hainanez cu orez pentru puiul hainanez 1 p...,0.712985
9,"The weather has been quite chilly lately, and ...",Tofună cu tofu cu Kimchi\n\nIngrediente princi...,Tocană de Kimchi cu Tofu\n\nIngrediente princi...,tofună cu tofu cu kimchi ingrediente principal...,tocană de kimchi cu tofu ingrediente principal...,0.726883


In [ ]:
# Get minimum and maximum scores and their indices
min_score_index_R = df_romanian['METEOR_score'].idxmin()
max_score_index_R = df_romanian['METEOR_score'].idxmax()

# Extract corresponding sentences and scores
min_score_value_R = df_romanian.loc[min_score_index_R, 'METEOR_score']
max_score_value_R = df_romanian.loc[max_score_index_R, 'METEOR_score']

min_reference_R = df_romanian.loc[min_score_index_R, 'Google-Romanian']
min_hypothesis_R = df_romanian.loc[min_score_index_R, 'Gemini-Romanian']

max_reference_R = df_romanian.loc[max_score_index_R, 'Google-Romanian']
max_hypothesis_R = df_romanian.loc[max_score_index_R, 'Gemini-Romanian']

# Print results
print(f"Minimum METEOR score: {min_score_value_R:.4f}")
print(f"  - Reference: {min_reference_R}")
print(f"  - Hypothesis: {min_hypothesis_R}")

print(f"\nMaximum METEOR score: {max_score_value_R:.4f}")
print(f"  - Reference: {max_reference_R}")
print(f"  - Hypothesis: {max_hypothesis_R}")

# Calculate average METEOR score
average_meteor_score_R = df_romanian['METEOR_score'].mean()
median_meteor_R = df_romanian['METEOR_score'].median()

print(f"\nThe Average METEOR of translating Romanian is: {average_meteor_score_R*100:.3f}%")
print(f"The Median METEOR score is: {median_meteor_R*100:.3f}%")

Minimum METEOR score: 0.5580
  - Reference: supa de pui cu taitei bulion puteți folosi bulion de pui sau bulion de legume cumpărat din magazin sau puteți să vă faceți singur pentru o supă mai aromată pui pieptul sau pulpele de pui fiert și mărunțit funcționează bine dacă nu aveți pui gătit puteți bracona niște piept sau pulpe de pui în bulion în timp ce faceți supa legume morcovi tocati telina taiata cubulete ceapa tocata optional taitei taiteii cu ou sunt clasici dar puteti folosi si alte tipuri de taitei cum ar fi taiteii de orez sau pastina condimente sare piper ierburi uscate precum cimbru sau oregano ingrediente optionale usturoi proaspăt tocat ghimbir proaspăt răzuit frunza de dafin pătrunjel proaspăt tocat pentru garnitură
  - Hypothesis:  supă de pui cu tăiței supă poți folosi supă de pui sau supă de legume cumpărată din magazin sau poți săți prepari propria supă pentru o aromă mai consistentă pui puiul fiert și sfărâmat fie din piept fie din pulpe merge bine dacă nu ai pui fie

# Part II: Chinese

In [ ]:
# Load data and handle empty rows
df_chinese = pd.read_csv("METEOR_Chinese.csv")
df_chinese = df_chinese.dropna()

# Preview data
df_chinese.head()

,Prompt,Google Translate - Chinese,Eng - Chinese (Gemini)
0,Suggest the ingredients needed for Roasted Swe...,蔬菜：\n\n2-3 個中等大小的紅薯（取決於大小）\n1 杯（約 150 克）新鮮蘑菇（例...,食材\n\n2-3 顆中等大小的地瓜\n1 杯（約 150 克）新鮮蘑菇（例如蠔菇、香菇或舞...
1,"Create an ingredient list in English, Chinese,...",櫛瓜和乳清乾酪油條配辣番茄醬（義大利南部）\n原料：\n\n1 個中等大小的西葫蘆（黃色或綠...,南義大利夏季西葫蘆乳清起士餅配香辣番茄醬\n食材\n\n1 個中等大小的西葫蘆（黃色或綠色）...
2,I want to go to the farmers market for a two c...,春菜湯\n\n2湯匙橄欖油\n1 顆中等大小的洋蔥，切丁\n2 根胡蘿蔔，切丁\n2 根芹菜...,春季蔬菜湯\n\n2 湯匙橄欖油\n1 個中等大小的洋蔥，切丁\n2 根胡蘿蔔，切丁\n2 ...
3,Tomorrow is Christmas Eve. What ingredients sh...,節日火雞佐蔓越莓醬炒（英文）\n\n原料：\n\n1磅碎火雞\n1大匙橄欖油\n1 顆中等大...,節日火雞翻炒配蔓越莓醬\n\n食材\n\n半公斤火雞絞肉\r\n1 湯匙橄欖油\r\n1 個...
4,What ingredients do I need for a dish typical ...,主菜：\n\n土耳其：這是節目的明星！ 為您的客人選擇大小合適的整隻火雞。\n餡料/調味品：...,主菜:\n\n火雞 : 這感恩節大餐的主角！ 選擇一個適合用餐人數大小的整隻火雞。\n填充物...


In [ ]:
# Select the columns containing the references and hypotheses (text to evaluate)
df_chinese['Google-Chinese'] = df_chinese['Google Translate - Chinese']
df_chinese['Gemini-Chinese'] = df_chinese['Eng - Chinese (Gemini)']

# Preview first few rows
df_chinese[:5]

,Prompt,Google Translate - Chinese,Eng - Chinese (Gemini),Google-Chinese,Gemini-Chinese
0,Suggest the ingredients needed for Roasted Swe...,蔬菜：\n\n2-3 個中等大小的紅薯（取決於大小）\n1 杯（約 150 克）新鮮蘑菇（例...,食材\n\n2-3 顆中等大小的地瓜\n1 杯（約 150 克）新鮮蘑菇（例如蠔菇、香菇或舞...,蔬菜：\n\n2-3 個中等大小的紅薯（取決於大小）\n1 杯（約 150 克）新鮮蘑菇（例...,食材\n\n2-3 顆中等大小的地瓜\n1 杯（約 150 克）新鮮蘑菇（例如蠔菇、香菇或舞...
1,"Create an ingredient list in English, Chinese,...",櫛瓜和乳清乾酪油條配辣番茄醬（義大利南部）\n原料：\n\n1 個中等大小的西葫蘆（黃色或綠...,南義大利夏季西葫蘆乳清起士餅配香辣番茄醬\n食材\n\n1 個中等大小的西葫蘆（黃色或綠色）...,櫛瓜和乳清乾酪油條配辣番茄醬（義大利南部）\n原料：\n\n1 個中等大小的西葫蘆（黃色或綠...,南義大利夏季西葫蘆乳清起士餅配香辣番茄醬\n食材\n\n1 個中等大小的西葫蘆（黃色或綠色）...
2,I want to go to the farmers market for a two c...,春菜湯\n\n2湯匙橄欖油\n1 顆中等大小的洋蔥，切丁\n2 根胡蘿蔔，切丁\n2 根芹菜...,春季蔬菜湯\n\n2 湯匙橄欖油\n1 個中等大小的洋蔥，切丁\n2 根胡蘿蔔，切丁\n2 ...,春菜湯\n\n2湯匙橄欖油\n1 顆中等大小的洋蔥，切丁\n2 根胡蘿蔔，切丁\n2 根芹菜...,春季蔬菜湯\n\n2 湯匙橄欖油\n1 個中等大小的洋蔥，切丁\n2 根胡蘿蔔，切丁\n2 ...
3,Tomorrow is Christmas Eve. What ingredients sh...,節日火雞佐蔓越莓醬炒（英文）\n\n原料：\n\n1磅碎火雞\n1大匙橄欖油\n1 顆中等大...,節日火雞翻炒配蔓越莓醬\n\n食材\n\n半公斤火雞絞肉\r\n1 湯匙橄欖油\r\n1 個...,節日火雞佐蔓越莓醬炒（英文）\n\n原料：\n\n1磅碎火雞\n1大匙橄欖油\n1 顆中等大...,節日火雞翻炒配蔓越莓醬\n\n食材\n\n半公斤火雞絞肉\r\n1 湯匙橄欖油\r\n1 個...
4,What ingredients do I need for a dish typical ...,主菜：\n\n土耳其：這是節目的明星！ 為您的客人選擇大小合適的整隻火雞。\n餡料/調味品：...,主菜:\n\n火雞 : 這感恩節大餐的主角！ 選擇一個適合用餐人數大小的整隻火雞。\n填充物...,主菜：\n\n土耳其：這是節目的明星！ 為您的客人選擇大小合適的整隻火雞。\n餡料/調味品：...,主菜:\n\n火雞 : 這感恩節大餐的主角！ 選擇一個適合用餐人數大小的整隻火雞。\n填充物...


In [ ]:
# Import jieba to perform text segmentation on Chinese text
!pip install jieba
import jieba

In [ ]:
# Define a new clean_text function that is specific for Chinese
def clean_text_C(text):
  # Remove newlines
  text_without_newline = re.sub('\n', ' ', text)

  # Remove all punctuations, including parentheses, colons, and others
  text_without_brackets = re.sub(r'[^\w\s]', '', text_without_newline)

  # Replace multiple consecutive spaces with a single space
  text_cleaned = re.sub(r'\s{2,}', ' ', text_without_brackets)

  # Tokenize chinese
  tokens = jieba.cut(text)

  # Join the tokens back into strings
  cleaned_text = ' '.join(token for token in tokens if token.strip())

  # Return the cleaned, tokenized tezt
  return cleaned_text

In [ ]:
# Clean the Chinese translations using clean_text
df_chinese['Google-Chinese'] = df_chinese['Google-Chinese'].apply(clean_text_C)
df_chinese['Gemini-Chinese'] = df_chinese['Gemini-Chinese'].apply(clean_text_C)

# Print first Chinese translations side by side to check
df_chinese.head()

,Prompt,Google Translate - Chinese,Eng - Chinese (Gemini),Google-Chinese,Gemini-Chinese
0,Suggest the ingredients needed for Roasted Swe...,蔬菜：\n\n2-3 個中等大小的紅薯（取決於大小）\n1 杯（約 150 克）新鮮蘑菇（例...,食材\n\n2-3 顆中等大小的地瓜\n1 杯（約 150 克）新鮮蘑菇（例如蠔菇、香菇或舞...,蔬菜 ： 2 - 3 個 中等 大小 的 紅薯 （ 取決 於 大小 ） 1 杯 （ 約 15...,食材 2 - 3 顆 中等 大小 的 地瓜 1 杯 （ 約 150 克 ） 新鮮 蘑菇 （ ...
1,"Create an ingredient list in English, Chinese,...",櫛瓜和乳清乾酪油條配辣番茄醬（義大利南部）\n原料：\n\n1 個中等大小的西葫蘆（黃色或綠...,南義大利夏季西葫蘆乳清起士餅配香辣番茄醬\n食材\n\n1 個中等大小的西葫蘆（黃色或綠色）...,櫛瓜 和 乳清 乾酪 油條 配辣 番茄 醬 （ 義 大利 南部 ） 原料 ： 1 個 中等 ...,南義 大利 夏季 西葫蘆 乳清 起士 餅配 香辣 番茄 醬 食材 1 個 中等 大小 的 西...
2,I want to go to the farmers market for a two c...,春菜湯\n\n2湯匙橄欖油\n1 顆中等大小的洋蔥，切丁\n2 根胡蘿蔔，切丁\n2 根芹菜...,春季蔬菜湯\n\n2 湯匙橄欖油\n1 個中等大小的洋蔥，切丁\n2 根胡蘿蔔，切丁\n2 ...,春菜湯 2 湯匙 橄欖油 1 顆 中等 大小 的 洋蔥 ， 切丁 2 根胡蘿 蔔 ， 切丁 ...,春季 蔬菜 湯 2 湯匙 橄欖油 1 個 中等 大小 的 洋蔥 ， 切丁 2 根胡蘿 蔔 ，...
3,Tomorrow is Christmas Eve. What ingredients sh...,節日火雞佐蔓越莓醬炒（英文）\n\n原料：\n\n1磅碎火雞\n1大匙橄欖油\n1 顆中等大...,節日火雞翻炒配蔓越莓醬\n\n食材\n\n半公斤火雞絞肉\r\n1 湯匙橄欖油\r\n1 個...,節日 火雞佐蔓 越莓 醬 炒 （ 英文 ） 原料 ： 1 磅 碎 火雞 1 大匙 橄欖油 1...,節日 火雞 翻炒 配蔓 越莓 醬 食材 半公斤 火雞 絞肉 1 湯匙 橄欖油 1 個 中等 ...
4,What ingredients do I need for a dish typical ...,主菜：\n\n土耳其：這是節目的明星！ 為您的客人選擇大小合適的整隻火雞。\n餡料/調味品：...,主菜:\n\n火雞 : 這感恩節大餐的主角！ 選擇一個適合用餐人數大小的整隻火雞。\n填充物...,主菜 ： 土耳其 ： 這是節 目的 明星 ！ 為 您 的 客人 選擇 大小 合適 的 整 隻...,主菜 : 火雞 : 這 感恩 節 大餐 的 主角 ！ 選擇 一個 適合 用餐 人數 大小 的...


In [ ]:
# Define a function that tokenizes and calculates METEOR scores for each row
def calculate_meteor_C(row):
    # Tokenize the hypothesis text
    hypothesis_tokens = word_tokenize(row['Gemini-Chinese'])
    # Tokenize the reference text
    reference_tokens = [word_tokenize(row['Google-Chinese'])]
    # Calculate and return METEOR scores
    return meteor_score(reference_tokens, hypothesis_tokens)

# Calculate METEOR score by applying the calculate_meteor function
df_chinese['METEOR_score'] = df_chinese.apply(calculate_meteor_C, axis=1)

# Return dataframe to see the newly added METEOR score
df_chinese


,Prompt,Google Translate - Chinese,Eng - Chinese (Gemini),Google-Chinese,Gemini-Chinese,METEOR_score
0,Suggest the ingredients needed for Roasted Swe...,蔬菜：\n\n2-3 個中等大小的紅薯（取決於大小）\n1 杯（約 150 克）新鮮蘑菇（例...,食材\n\n2-3 顆中等大小的地瓜\n1 杯（約 150 克）新鮮蘑菇（例如蠔菇、香菇或舞...,蔬菜 ： 2 - 3 個 中等 大小 的 紅薯 （ 取決 於 大小 ） 1 杯 （ 約 15...,食材 2 - 3 顆 中等 大小 的 地瓜 1 杯 （ 約 150 克 ） 新鮮 蘑菇 （ ...,0.748820
1,"Create an ingredient list in English, Chinese,...",櫛瓜和乳清乾酪油條配辣番茄醬（義大利南部）\n原料：\n\n1 個中等大小的西葫蘆（黃色或綠...,南義大利夏季西葫蘆乳清起士餅配香辣番茄醬\n食材\n\n1 個中等大小的西葫蘆（黃色或綠色）...,櫛瓜 和 乳清 乾酪 油條 配辣 番茄 醬 （ 義 大利 南部 ） 原料 ： 1 個 中等 ...,南義 大利 夏季 西葫蘆 乳清 起士 餅配 香辣 番茄 醬 食材 1 個 中等 大小 的 西...,0.661223
2,I want to go to the farmers market for a two c...,春菜湯\n\n2湯匙橄欖油\n1 顆中等大小的洋蔥，切丁\n2 根胡蘿蔔，切丁\n2 根芹菜...,春季蔬菜湯\n\n2 湯匙橄欖油\n1 個中等大小的洋蔥，切丁\n2 根胡蘿蔔，切丁\n2 ...,春菜湯 2 湯匙 橄欖油 1 顆 中等 大小 的 洋蔥 ， 切丁 2 根胡蘿 蔔 ， 切丁 ...,春季 蔬菜 湯 2 湯匙 橄欖油 1 個 中等 大小 的 洋蔥 ， 切丁 2 根胡蘿 蔔 ，...,0.807811
3,Tomorrow is Christmas Eve. What ingredients sh...,節日火雞佐蔓越莓醬炒（英文）\n\n原料：\n\n1磅碎火雞\n1大匙橄欖油\n1 顆中等大...,節日火雞翻炒配蔓越莓醬\n\n食材\n\n半公斤火雞絞肉\r\n1 湯匙橄欖油\r\n1 個...,節日 火雞佐蔓 越莓 醬 炒 （ 英文 ） 原料 ： 1 磅 碎 火雞 1 大匙 橄欖油 1...,節日 火雞 翻炒 配蔓 越莓 醬 食材 半公斤 火雞 絞肉 1 湯匙 橄欖油 1 個 中等 ...,0.815077
4,What ingredients do I need for a dish typical ...,主菜：\n\n土耳其：這是節目的明星！ 為您的客人選擇大小合適的整隻火雞。\n餡料/調味品：...,主菜:\n\n火雞 : 這感恩節大餐的主角！ 選擇一個適合用餐人數大小的整隻火雞。\n填充物...,主菜 ： 土耳其 ： 這是節 目的 明星 ！ 為 您 的 客人 選擇 大小 合適 的 整 隻...,主菜 : 火雞 : 這 感恩 節 大餐 的 主角 ！ 選擇 一個 適合 用餐 人數 大小 的...,0.679456
5,Design an ingredient list for a hearty Irish s...,豐盛的愛爾蘭燉菜\n\n原料：\n\n1大匙橄欖油\n1 顆大洋蔥，切丁\n2根大胡蘿蔔，去...,燉菜\n\n食材:\n\n1 湯匙橄欖油\n1 個大洋蔥，切丁\n2 大胡蘿蔔，去皮切丁\n...,豐盛 的 愛爾蘭燉 菜 原料 ： 1 大匙 橄欖油 1 顆 大洋 蔥 ， 切丁 2 根大 胡...,燉 菜 食材 : 1 湯匙 橄欖油 1 個 大洋 蔥 ， 切丁 2 大 胡蘿 蔔 ， 去皮 ...,0.748676
6,Create a recipe for a Pad Thai using only ingr...,簡易泰式炒麵\n\n原料：\n\n1大匙植物油\n2 青蔥，切成薄片\n2 顆雞蛋，打散\n...,泰式炒河粉\n食材:\n\n1 湯匙植物油\n2 顆紅蔥，切薄片\n2 顆雞蛋，打散\n15...,簡易 泰式 炒 麵 原料 ： 1 大匙 植物油 2 青蔥 ， 切成 薄片 2 顆雞蛋 ， 打...,泰式 炒河粉 食材 : 1 湯匙 植物油 2 顆紅蔥 ， 切 薄片 2 顆雞蛋 ， 打散 1...,0.679838
7,"Recently, I just finished strolling through th...",臭豆腐：\n煎炸用植物油\n玉米澱粉（可選，用於塗層）\n醃製蔬菜（高麗菜絲、蘿蔔絲等）\n...,臭豆腐：\n用於油炸的植物油\n玉米澱粉（可選，用於塗層）\n醃製蔬菜（切碎的捲心菜、蘿蔔等...,臭豆腐 ： 煎炸 用 植物油 玉米 澱 粉 （ 可選 ， 用 於 塗層 ） 醃 製 蔬菜 （...,臭豆腐 ： 用 於 油炸 的 植物油 玉米 澱 粉 （ 可選 ， 用 於 塗層 ） 醃 製 ...,0.877264
8,My Singaporean friend is coming to visit me at...,海南雞飯\n\n海南雞：\n1隻整隻雞（約1.7公斤），室溫\n5公分新鮮生薑，未剝皮\n2...,海南雞飯\n\n海南雞部分：\n1 隻全雞（約 1.7 公斤），室溫\n5 公分生薑，不去皮...,海南 雞飯 海南 雞 ： 1 隻 整 隻 雞 （ 約 1.7 公斤 ） ， 室溫 5 公分 ...,海南 雞飯 海南 雞 部分 ： 1 隻 全雞 （ 約 1.7 公斤 ） ， 室溫 5 公分 ...,0.761140
9,"The weather has been quite chilly lately, and ...",泡菜豆腐燉菜\n\n主要成分：\n\n1塊（約350克）硬豆腐，瀝乾並切成一口大小的方塊\n...,泡菜嫩豆腐煲\n\n主要食材:\n\n1 塊（約 350 克）嫩豆腐，瀝乾水分，切成小塊\n...,泡菜 豆腐 燉 菜 主要 成分 ： 1 塊 （ 約 350 克 ） 硬 豆腐 ， 瀝乾並 切...,泡菜 嫩豆腐 煲 主要 食材 : 1 塊 （ 約 350 克 ） 嫩豆腐 ， 瀝乾 水分 ，...,0.730320


In [ ]:
# Get minimum and maximum scores and their indices
min_score_index_C = df_chinese['METEOR_score'].idxmin()
max_score_index_C = df_chinese['METEOR_score'].idxmax()

# Extract corresponding sentences and scores
min_score_value_C = df_chinese.loc[min_score_index_C, 'METEOR_score']
max_score_value_C = df_chinese.loc[max_score_index_C, 'METEOR_score']

min_reference_C = df_chinese.loc[min_score_index_C, 'Google-Chinese']
min_hypothesis_C = df_chinese.loc[min_score_index_C, 'Gemini-Chinese']

max_reference_C = df_chinese.loc[max_score_index_C, 'Google-Chinese']
max_hypothesis_C = df_chinese.loc[max_score_index_C, 'Gemini-Chinese']

# Print results
print(f"Minimum METEOR score: {min_score_value_C:.4f}")
print(f"  - Reference: {min_reference_C}")
print(f"  - Hypothesis: {min_hypothesis_C}")

print(f"\nMaximum METEOR score: {max_score_value_C:.4f}")
print(f"  - Reference: {max_reference_C}")
print(f"  - Hypothesis: {max_hypothesis_C}")

# Calculate average METEOR score
average_meteor_score_C = df_chinese['METEOR_score'].mean()
median_meteor_C = df_chinese['METEOR_score'].median()

print(f"\nThe Average METEOR of translating Chinese is: {average_meteor_score_C*100:.3f}%")
print(f"The Median METEOR score is: {median_meteor_C*100:.3f}%")

Minimum METEOR score: 0.5547
  - Reference: 经典 巧克力 饼干 原料 ： 干 成分 ： 1 杯 （ 2 支 ） 无盐 黄油 ， 软化 1 杯 砂糖 ½ 杯 包装 淡 红糖 2 个 大 鸡蛋 2 3 / 4 杯 通用 面粉 1 茶匙 小苏打 1 茶匙 盐 湿 成分 ： 2 茶匙 纯 香草 精 巧克力 脆片 ： 2 杯半甜 巧克力 片 （ 或 牛奶 、 黑 巧克力 或 白巧克力 片 的 混合物 ）
  - Hypothesis: 经典 巧克力 曲奇饼 原料 : 干性 原料 : 1 杯 ( 2 根 ) 无盐 黄油 ， 软化 1 杯 细砂糖 ½ 杯 包装 浅棕色 糖 2 个 鸡蛋 2 ¾ 杯 通用 面粉 1 茶匙 小 苏打粉 1 茶匙 盐 湿性 原料 : 2 茶匙 纯 香草 精 巧克力 芯片 : 2 杯 半甜 巧克力 芯片 （ 或 混合 牛奶 、 黑 巧克力 或 白色 巧克力 芯片 ）

Maximum METEOR score: 0.9936
  - Reference: 印度 綠 咖哩 綠色 調味醬 ： 1 把 新鮮 香菜 葉 ½ 杯 新鮮 薄荷 葉 1 - 2 個青 辣椒 （ 依 香料 偏好 調整 ） 1 個 小洋 蔥 ， 切碎 1 英寸 生 薑 ， 去皮 切碎 3 瓣 大蒜 ， 切碎 1 湯匙 綠 荳 蔻 莢 （ 或 ½ 茶匙 磨 荳 蔻 ） 1 茶匙 小茴香 籽 1 茶匙 芫荽 籽 ½ 茶匙 薑 黃 粉 咖哩 部分 ： 1 湯匙 植物油 1 片 月桂 葉 （ 可選 ） 1 茶匙 小茴香 籽 1 個 中等 大小 的 洋蔥 ， 切片 400 克去 骨 去皮 雞 胸肉 或 大腿肉 ， 切成 一口 大小 的 塊 狀 （ 也 可以 用 豆腐 代替 ， 適合 素食者 ） 1 罐 ( 400 毫升 ) 椰奶 1 杯 蔬菜 湯 1 湯匙 番茄 醬 （ 或稱 番茄 沙司 ） 1 湯匙 咖哩粉 （ 可選 ） 鹽 ( 適量 ) 新鮮 香菜 （ 可選 ， 用 於 裝飾 ） 煮熟 的 米 飯 或 烤餅 ( 用 於 搭配 )
  - Hypothesis: 印度 綠 咖哩 綠色 調味醬 ： 1 把 新鮮 香菜 葉 ½ 杯 新鮮 薄荷 葉 1 - 2 個青 辣椒 （ 依 香料 偏好 調整 ） 1 個 小洋 蔥 ， 切碎 1 吋 生 薑 ， 去皮 切碎

# Part III: Greek

In [ ]:
# Load data and handle empty rows
df_greek = pd.read_csv("METEOR_Greek.csv")
df_greek = df_greek.dropna()

# Preview data
df_greek.head()

,Prompt,Google Translate,Eng - Greek (Gemini)
0,Suggest the ingredients needed for Roasted Swe...,Λαχανικά:\n\n2-3 μέτριες γλυκοπατάτες (ανάλογα...,Λαχανικά:\n\n2-3 μεσαίες γλυκοπατάτες (ανάλογα...
1,"Create an ingredient list in English, Chinese,...",Καλοκαιρινό σκουός και τηγανητές ρικότα με πικ...,Φρίτετες Κολοκυθάκια με Ρικότα και Πικάντικη Σ...
2,I want to go to the farmers market for a two c...,Ανοιξιάτικη σούπα λαχανικών\nΥλικά:\n\n2 κουτα...,Ανοιξιάτικη Σούπα Λαχανικών \nΥλικά:\n\n2 κουτ...
3,Tomorrow is Christmas Eve. What ingredients sh...,Γιορτινό Stir-Fry γαλοπούλας με σάλτσα Cranber...,Γιορτινό Στιρ-Φράι με Γαλοπούλα και Σάλτσα με ...
4,What ingredients do I need for a dish typical ...,Κύριο πιάτο:\n\nΤουρκία: Αυτή είναι η σταρ του...,Κυρίως Πιάτο \n\nΓαλοπούλα : Αυτή είναι η βασί...


In [ ]:
# Select the columns containing the references and hypotheses (text to evaluate)
df_greek['Google-Greek'] = df_greek['Google Translate']
df_greek['Gemini-Greek'] = df_greek['Eng - Greek (Gemini)']

# Preview first few rows
df_greek[:5]

,Prompt,Google Translate,Eng - Greek (Gemini),Google-Greek,Gemini-Greek
0,Suggest the ingredients needed for Roasted Swe...,Λαχανικά:\n\n2-3 μέτριες γλυκοπατάτες (ανάλογα...,Λαχανικά:\n\n2-3 μεσαίες γλυκοπατάτες (ανάλογα...,Λαχανικά:\n\n2-3 μέτριες γλυκοπατάτες (ανάλογα...,Λαχανικά:\n\n2-3 μεσαίες γλυκοπατάτες (ανάλογα...
1,"Create an ingredient list in English, Chinese,...",Καλοκαιρινό σκουός και τηγανητές ρικότα με πικ...,Φρίτετες Κολοκυθάκια με Ρικότα και Πικάντικη Σ...,Καλοκαιρινό σκουός και τηγανητές ρικότα με πικ...,Φρίτετες Κολοκυθάκια με Ρικότα και Πικάντικη Σ...
2,I want to go to the farmers market for a two c...,Ανοιξιάτικη σούπα λαχανικών\nΥλικά:\n\n2 κουτα...,Ανοιξιάτικη Σούπα Λαχανικών \nΥλικά:\n\n2 κουτ...,Ανοιξιάτικη σούπα λαχανικών\nΥλικά:\n\n2 κουτα...,Ανοιξιάτικη Σούπα Λαχανικών \nΥλικά:\n\n2 κουτ...
3,Tomorrow is Christmas Eve. What ingredients sh...,Γιορτινό Stir-Fry γαλοπούλας με σάλτσα Cranber...,Γιορτινό Στιρ-Φράι με Γαλοπούλα και Σάλτσα με ...,Γιορτινό Stir-Fry γαλοπούλας με σάλτσα Cranber...,Γιορτινό Στιρ-Φράι με Γαλοπούλα και Σάλτσα με ...
4,What ingredients do I need for a dish typical ...,Κύριο πιάτο:\n\nΤουρκία: Αυτή είναι η σταρ του...,Κυρίως Πιάτο \n\nΓαλοπούλα : Αυτή είναι η βασί...,Κύριο πιάτο:\n\nΤουρκία: Αυτή είναι η σταρ του...,Κυρίως Πιάτο \n\nΓαλοπούλα : Αυτή είναι η βασί...


In [ ]:
# Clean the Greek translations using clean_text
df_greek['Google-Greek'] = df_greek['Google-Greek'].apply(clean_text)
df_greek['Gemini-Greek'] = df_greek['Gemini-Greek'].apply(clean_text)

# Print first Greek translations side by side to check
df_greek.head()

,Prompt,Google Translate,Eng - Greek (Gemini),Google-Greek,Gemini-Greek
0,Suggest the ingredients needed for Roasted Swe...,Λαχανικά:\n\n2-3 μέτριες γλυκοπατάτες (ανάλογα...,Λαχανικά:\n\n2-3 μεσαίες γλυκοπατάτες (ανάλογα...,λαχανικά 23 μέτριες γλυκοπατάτες ανάλογα με το...,λαχανικά 23 μεσαίες γλυκοπατάτες ανάλογα με το...
1,"Create an ingredient list in English, Chinese,...",Καλοκαιρινό σκουός και τηγανητές ρικότα με πικ...,Φρίτετες Κολοκυθάκια με Ρικότα και Πικάντικη Σ...,καλοκαιρινό σκουός και τηγανητές ρικότα με πικ...,φρίτετες κολοκυθάκια με ρικότα και πικάντικη σ...
2,I want to go to the farmers market for a two c...,Ανοιξιάτικη σούπα λαχανικών\nΥλικά:\n\n2 κουτα...,Ανοιξιάτικη Σούπα Λαχανικών \nΥλικά:\n\n2 κουτ...,ανοιξιάτικη σούπα λαχανικών υλικά 2 κουταλιές ...,ανοιξιάτικη σούπα λαχανικών υλικά 2 κουταλιές ...
3,Tomorrow is Christmas Eve. What ingredients sh...,Γιορτινό Stir-Fry γαλοπούλας με σάλτσα Cranber...,Γιορτινό Στιρ-Φράι με Γαλοπούλα και Σάλτσα με ...,γιορτινό stirfry γαλοπούλας με σάλτσα cranberr...,γιορτινό στιρφράι με γαλοπούλα και σάλτσα με κ...
4,What ingredients do I need for a dish typical ...,Κύριο πιάτο:\n\nΤουρκία: Αυτή είναι η σταρ του...,Κυρίως Πιάτο \n\nΓαλοπούλα : Αυτή είναι η βασί...,κύριο πιάτο τουρκία αυτή είναι η σταρ του σόου...,κυρίως πιάτο γαλοπούλα αυτή είναι η βασίλισσα ...


In [ ]:
# Define a function that tokenizes and calculates METEOR scores for each row
def calculate_meteor_G(row):
    # Tokenize the hypothesis tex
    hypothesis_tokens = word_tokenize(row['Gemini-Greek'])
    # Tokenize the reference text
    reference_tokens = [word_tokenize(row['Google-Greek'])]
    # Calculate and return METEOR
    return meteor_score(reference_tokens, hypothesis_tokens)

# Calculate METEOR score by applying the calculate_meteor function
df_greek['METEOR_score'] = df_greek.apply(calculate_meteor_G, axis=1)

# Return dataframe to see the newly added METEOR score
df_greek

,Prompt,Google Translate,Eng - Greek (Gemini),Google-Greek,Gemini-Greek,METEOR_score
0,Suggest the ingredients needed for Roasted Swe...,Λαχανικά:\n\n2-3 μέτριες γλυκοπατάτες (ανάλογα...,Λαχανικά:\n\n2-3 μεσαίες γλυκοπατάτες (ανάλογα...,λαχανικά 23 μέτριες γλυκοπατάτες ανάλογα με το...,λαχανικά 23 μεσαίες γλυκοπατάτες ανάλογα με το...,0.702895
1,"Create an ingredient list in English, Chinese,...",Καλοκαιρινό σκουός και τηγανητές ρικότα με πικ...,Φρίτετες Κολοκυθάκια με Ρικότα και Πικάντικη Σ...,καλοκαιρινό σκουός και τηγανητές ρικότα με πικ...,φρίτετες κολοκυθάκια με ρικότα και πικάντικη σ...,0.778683
2,I want to go to the farmers market for a two c...,Ανοιξιάτικη σούπα λαχανικών\nΥλικά:\n\n2 κουτα...,Ανοιξιάτικη Σούπα Λαχανικών \nΥλικά:\n\n2 κουτ...,ανοιξιάτικη σούπα λαχανικών υλικά 2 κουταλιές ...,ανοιξιάτικη σούπα λαχανικών υλικά 2 κουταλιές ...,0.615468
3,Tomorrow is Christmas Eve. What ingredients sh...,Γιορτινό Stir-Fry γαλοπούλας με σάλτσα Cranber...,Γιορτινό Στιρ-Φράι με Γαλοπούλα και Σάλτσα με ...,γιορτινό stirfry γαλοπούλας με σάλτσα cranberr...,γιορτινό στιρφράι με γαλοπούλα και σάλτσα με κ...,0.644951
4,What ingredients do I need for a dish typical ...,Κύριο πιάτο:\n\nΤουρκία: Αυτή είναι η σταρ του...,Κυρίως Πιάτο \n\nΓαλοπούλα : Αυτή είναι η βασί...,κύριο πιάτο τουρκία αυτή είναι η σταρ του σόου...,κυρίως πιάτο γαλοπούλα αυτή είναι η βασίλισσα ...,0.588020
5,Design an ingredient list for a hearty Irish s...,Χορταστικό ιρλανδικό στιφάδο\nΣυστατικά:\n\n1 ...,Αχνιστό Ιρλανδικό Στιφάδο\nΥλικά:\n\n1 κουταλι...,χορταστικό ιρλανδικό στιφάδο συστατικά 1 κουτα...,αχνιστό ιρλανδικό στιφάδο υλικά 1 κουταλιά σού...,0.624363
6,Create a recipe for a Pad Thai using only ingr...,Εύκολο Παντ Ταϊ \n\nΣυστατικά:\n\n1 κουταλιά τ...,Εύκολο Παντ Ταϊ \nΥλικά:\n\n1 κουταλιά σούπας ...,εύκολο παντ ταϊ συστατικά 1 κουταλιά της σούπα...,εύκολο παντ ταϊ υλικά 1 κουταλιά σούπας φυτικό...,0.615048
7,"Recently, I just finished strolling through th...",Βρωμερό τόφου:\nΦυτικό λάδι για τηγάνισμα\nΆμυ...,Μυρωδάτο Τυρί \nΦυτικό λάδι για τηγάνισμα\nΆμυ...,βρωμερό τόφου φυτικό λάδι για τηγάνισμα άμυλο ...,μυρωδάτο τυρί φυτικό λάδι για τηγάνισμα άμυλο ...,0.682947
8,My Singaporean friend is coming to visit me at...,Ρύζι κοτόπουλο Hainanese\n\nΓια το κοτόπουλο H...,Κοτόπουλο με Ρύζι αλά Χαϊλονάν \nΓια το Κοτόπο...,ρύζι κοτόπουλο hainanese για το κοτόπουλο hain...,κοτόπουλο με ρύζι αλά χαϊλονάν για το κοτόπουλ...,0.758523
9,"The weather has been quite chilly lately, and ...",Πικάντικο Π στιφάδο Κιμτσί με Τόφου\n\nΚύρια σ...,Πικάντικο Π στιφάδο Κιμτσί με Τόφου\nΚύρια Υλι...,πικάντικο π στιφάδο κιμτσί με τόφου κύρια συστ...,πικάντικο π στιφάδο κιμτσί με τόφου κύρια υλικ...,0.746999


In [ ]:
# Get minimum and maximum scores and their indices
min_score_index_G = df_greek['METEOR_score'].idxmin()
max_score_index_G = df_greek['METEOR_score'].idxmax()

# Extract corresponding sentences and scores
min_score_value_G = df_greek.loc[min_score_index_G, 'METEOR_score']
max_score_value_G = df_greek.loc[max_score_index_G, 'METEOR_score']

min_reference_G = df_greek.loc[min_score_index_G, 'Google-Greek']
min_hypothesis_G = df_greek.loc[min_score_index_G, 'Gemini-Greek']

max_reference_G = df_greek.loc[max_score_index_G, 'Google-Greek']
max_hypothesis_G = df_greek.loc[max_score_index_G, 'Gemini-Greek']

# Print results
print(f"Minimum METEOR score: {min_score_value_G:.4f}")
print(f"  - Reference: {min_reference_G}")
print(f"  - Hypothesis: {min_hypothesis_G}")

print(f"\nMaximum METEOR score: {max_score_value_G:.4f}")
print(f"  - Reference: {max_reference_G}")
print(f"  - Hypothesis: {max_hypothesis_G}")

# Calculate average METEOR score
average_meteor_score_G = df_greek['METEOR_score'].mean()
median_meteor_G = df_greek['METEOR_score'].median()

print(f"\nThe Average METEOR of translating Greek is: {average_meteor_score_G*100:.3f}%")
print(f"The Median METEOR score is: {median_meteor_G*100:.3f}%")

Minimum METEOR score: 0.5622
  - Reference: miso σάλτσα σόγιας ramen για το ζωμό 4 φλιτζάνια ζωμό κοτόπουλου ή λαχανικών 4 φλιτζάνια νερό 1 κουταλιά της σούπας mirin γλυκό κρασί από ρύζι 1 κουταλιά της σούπας σάκε προαιρετικά 23 κουταλιές της σούπας πάστα miso λευκή γλυκιά ή κόκκινη ανάλογα με τις προτιμήσεις σας φύκια kombu φύκια προαιρετικά για πρόσθετο βάθος γεύσης katsuobushi νιφάδες παλαμίδας φύκια προαιρετικά για γαρνίρισμα για το ramen 2 μερίδες φρέσκα noodles ramen ή αποξηραμένα noodles ramen σύμφωνα με τις οδηγίες της συσκευασίας κρεμμύδια σε φέτες για γαρνίρισμα προαιρετικές επικαλύψεις επιλέξτε οποιαδήποτε ή όλα χοιρινό chashu κοιλιά χοιρινή κοκκινιστή αυγό μαλακό μαριναρισμένο ή σκέτο σεντόνια από φύκια nori βλαστάρια μπαμπού menma καλαμπόκι βλαστάρια φασολιών narutomaki φέτες κέικ ψαριού καμαμπόκο
  - Hypothesis: μίσο ραμέν με σάλτσα σόγιας για το ζωμό 4 φλιτζάνια ζωμός κοτόπουλου ή λαχανικών 4 φλιτζάνια νερό 1 κουταλιά της σούπας μίριν γλυκό ρυζο κρασί 1 κουταλιά της σούπ

# Part IV: Comparison

In [ ]:
print(f"\nThe Average METEOR of translating Romanian is: {average_meteor_score_R*100:.3f}%")
print(f"The Median METEOR score is: {median_meteor_R*100:.3f}%")

print(f"\nThe Average METEOR of translating Chinese is: {average_meteor_score_C*100:.3f}%")
print(f"The Median METEOR score is: {median_meteor_C*100:.3f}%")

print(f"\nThe Average METEOR of translating Greek is: {average_meteor_score_G*100:.3f}%")
print(f"The Median METEOR score is: {median_meteor_G*100:.3f}%")


The Average METEOR of translating Romanian is: 69.545%
The Median METEOR score is: 69.798%

The Average METEOR of translating Chinese is: 73.036%
The Median METEOR score is: 73.313%

The Average METEOR of translating Greek is: 71.654%
The Median METEOR score is: 71.333%
